In [1]:
import findspark
import os
import configparser
findspark.init()
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from functools import reduce
from pyspark.sql import DataFrame
import pymongo

In [2]:

# os.environ['PYSPARK_PYTHON'] = '/usr/bin/python3'
# os.environ['PYSPARK_DRIVER_PYTHON'] = '/usr/bin/python3'



dataset1_path = './Dataset/datasets_894_813759_2015.csv'
dataset2_path = './Dataset/datasets_894_813759_2016.csv'
dataset3_path = './Dataset/datasets_894_813759_2017.csv'
dataset4_path = './Dataset/datasets_894_813759_2018.csv'
dataset5_path = './Dataset/sample.txt'



print(dataset1_path)

./Dataset/datasets_894_813759_2015.csv


In [3]:
sc = SparkSession.builder.config('spark.jars.packages',
                                    'org.apache.hadoop:hadoop-aws:2.6.4').getOrCreate()

# sc._jsc.hadoopConfiguration().set("fs.s3a.access.key",
#                                      os.environ['AWS_ACCESS_KEY_ID'])
# sc._jsc.hadoopConfiguration().set("fs.s3a.secret.key",
#                                      os.environ['AWS_SECRET_ACCESS_KEY'])
# sc._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

KeyboardInterrupt: 

In [3]:
from pyspark import SparkConf, SparkContext
conf = (SparkConf()
         .setMaster("local")
         .setAppName("My app")
         .set("spark.executor.memory", "1g"))
sc = SparkContext(conf = conf)

In [4]:
_matchscore = 1
_missscore   = 0
_firstgappenalty = -2
_furthergappentaly = -1
_topN = 20
_minScoreToPrint = 1

_gap = "-"
_totalPairsEvaluated =0

In [5]:
def _GetScoreOnePair (s1, s2 ):
    global _totalPairsEvaluated

    thisscore = 0
    firstgap = False
    
    m = min(len(s1), len(s2))
    for pos in range (0, m):
        if (s1[pos] == s2[pos]) and (s2[pos] != _gap) :
            thisscore += _matchscore
            firstgap = False
        elif (s1[pos] == _gap) and ( s2[pos] != _gap ) :
            if firstgap == True:
                thisscore += _furthergappentaly
            else:
                firstgap = True
                thisscore += _firstgappenalty
        elif (s2[pos] == _gap) and ( s1[pos] != _gap ) :
            if firstgap == True:
                thisscore += _furthergappentaly
            else:
                firstgap = True
                thisscore += _firstgappenalty
        else :
            firstgap = False
            thisscore += _missscore

    _totalPairsEvaluated = _totalPairsEvaluated + 1
    if ( _totalPairsEvaluated%100000) == 0 :
        print (_totalPairsEvaluated)
        
    return thisscore

In [6]:
def  _OutputProteinPairWithScore (_sl1, _sl2, score):
    str1 = _sl1.rstrip()
    str2 = _sl2.rstrip()
    m = max(len(str1), len(str2));


    if score >= _minScoreToPrint:
        print ("pos", end = " " )
        for pos in range (0, m):
            print(pos, end = " ")
        print("  score = ", score)

        print ("   ", end = " " )
        for pos in range (0, len(str1)):
            print (str1[pos], end = " " )
        print("")

        print ("   ", end = " " )
        for pos in range (0, len(str2)):
            print (str2[pos], end = " " )
        print("")
        print("")

In [7]:
def _CreateProteinVariantsStartingSpaces ( _s1, _diff_len ):
    result = []
    result.append(_s1)
    
    for i in range (1, _diff_len):
        s = _s1.rjust(len(_s1)+i)
        result.append(s)
    return result


In [8]:
def _CreateProteinVariantsWithGap (_sl1, _num_gaps, _max_num_gaps, _min_index, _max_index ):
    strings = []
    if  _num_gaps == _max_num_gaps:
        strings.append(_sl1)
        
    strinlen = len(_sl1)
    if strinlen >  _max_index:
        strinlen =  _max_index
        
    #do not add a gap at the beginning and the end    
    for i in range (_min_index+1, strinlen-1):
        stl = _sl1[0:i] + _gap + _sl1[i:];
        strings.append(stl)
        #print (_min_index, _max_index, stl)
        if  _num_gaps > 1 :
            stl2 = _CreateProteinVariantsWithGap(stl, _num_gaps-1, _max_num_gaps,  _min_index, _max_index )
            for s in stl2:
                strings.append(s)
            
    return strings

In [9]:
def _GetScoresWithVariants ( _sl1, _sl2, max_gaps ):
    currentmin = -1000 # Current minimum value to be added to the vec
    currentmax = -1000 # Maximum sore observed so far
    scorevec = []
    s1vec    = []
    
    s1 = _sl1.upper()
    s2 = _sl2.upper()


    diff_len = len(s2) - len(s1)
    if diff_len < 0 :
        diff_len = 0
    s1x = _CreateProteinVariantsStartingSpaces (s1, diff_len )
    for s in s1x:
        max_len = min (len(s), len(s2)) + max_gaps
        min_index = 0
        for j in range (0, len(s) ):
            if s[j] == " ":
                min_index = min_index + 1
            else:
                break

        s1var = _CreateProteinVariantsWithGap(s, max_gaps, max_gaps, min_index, max_len )
        for sx in s1var:
            s1vec.append(sx)
            
    #String s2 is assumed to be from a database, so we do not calculate variants on it

    #remove duplicates
    s1variants = list(dict.fromkeys(s1vec))
    i = 0
    for s1var in s1variants:
        score = _GetScoreOnePair (s1var, s2 )
        tup = (score, s1var, s2)
        if i < _topN:
            scorevec.append(tup)
            if  i == 0:
                currentmin = score
            elif  score < currentmin:
                currentmin = score
                if i == 0 :
                    currentmax = score
                elif score > currentmax:
                    currentmax = score
        else:
            if  score > currentmin:
                scorevec.append(tup)
        i = i+1
        scorevec.sort(key = lambda x : -x[0]);
        scorevec = scorevec[0:_topN]
        currentmin = min (scorevec)[0]
        currentmax = max (scorevec)[0]
            
    return scorevec

In [10]:
sc

<SparkContext master=local appName=My app>

In [13]:
print("Usage : python3 proteinscoring-with-test.py proteinfile.txt dbfile.txt max_gaps")
allvec = []

num_entries_in_db = 0

# var_pro=protein_seq.collect()
#dbfile = sys.argv[2] old  code
#dbfile = sc.textFile("./dbnew.txt")
dbfile=sc.parallelize(['abcdef','ghijk','abcdeg'])
# dbfile.cache()
dbsample = dbfile.map(lambda w : ('abc',w,2))
# dbsample1=dbsample.map(lambda w:(w,max_gaps))
max_gaps = 2
#dbsample1.collect()
#     for dbsequence in open(dbfile, 'r').readlines():  old code
#         num_entries_in_db = num_entries_in_db + 1    olld code
#scorevec = _GetScoresWithVariants ( proteinsequence, dbsequence, max_gaps) # old code
#scorevec=dbsample.map(lambda x:  _GetScoresWithVariants(x[0],x[1],x[2]))
scorevec=dbsample.flatMap(lambda x:  _GetScoresWithVariants(x[0],x[1],x[2]))
#print("scorevec obtained ",scorevec)
#     for s in scorevec:     old  code not needed
#         allvec.append(s)


#     allvec.sort (key = lambda x : -x[0] ) old code
#     allvec = allvec [0:_topN]    old code
#  scorevec.sort (key = lambda x : -x[0] ) 
# scorevec = scorevec [0:_topN] 

#for tup in allvec: old code
#     for tup in scorevec:    old  code

#         _OutputProteinPairWithScore (tup[1], tup[2], tup[0])  old code
#     finalans=scorevec.map(lambda y:_OutputProteinPairWithScore (y[1], y[2], y[0]))   
#print("Number of entries in database " , num_entries_in_db)    
print("Total Pairs evaluated " , _totalPairsEvaluated) 
#print("final ans",finalans)

Usage : python3 proteinscoring-with-test.py proteinfile.txt dbfile.txt max_gaps
Total Pairs evaluated  0


In [12]:
scorevec.collect()

[(3, 'ABC', 'ABCDEF'),
 (0, ' ABC', 'ABCDEF'),
 (0, '  ABC', 'ABCDEF'),
 (-1, 'A-BC', 'ABCDEF'),
 (-2, 'A--BC', 'ABCDEF'),
 (-2, ' A-BC', 'ABCDEF'),
 (-2, '  A-BC', 'ABCDEF'),
 (-3, ' A--BC', 'ABCDEF'),
 (-3, '  A--BC', 'ABCDEF'),
 (0, 'ABC', 'GHIJK'),
 (0, ' ABC', 'GHIJK'),
 (-2, 'A-BC', 'GHIJK'),
 (-2, ' A-BC', 'GHIJK'),
 (-3, 'A--BC', 'GHIJK'),
 (-3, ' A--BC', 'GHIJK'),
 (3, 'ABC', 'ABCDEG'),
 (0, ' ABC', 'ABCDEG'),
 (0, '  ABC', 'ABCDEG'),
 (-1, 'A-BC', 'ABCDEG'),
 (-2, 'A--BC', 'ABCDEG'),
 (-2, ' A-BC', 'ABCDEG'),
 (-2, '  A-BC', 'ABCDEG'),
 (-3, ' A--BC', 'ABCDEG'),
 (-3, '  A--BC', 'ABCDEG')]

In [14]:
scorevec.saveAsTextFile("./results_ssds")

In [8]:
RDD = sc.parallelize([1,2,3,4,5,6])

In [11]:
RDD
RDD.saveAsTextFile("./results_so")

Py4JJavaError: An error occurred while calling o68.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:100)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:385)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDDFu

In [10]:
RDD.saveAsTextFile("./results123")

Py4JJavaError: An error occurred while calling o57.saveAsTextFile.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.internal.io.SparkHadoopWriter$.write(SparkHadoopWriter.scala:100)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply$mcV$sp(PairRDDFunctions.scala:1096)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopDataset$1.apply(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:112)
	at org.apache.spark.rdd.RDD.withScope(RDD.scala:385)
	at org.apache.spark.rdd.PairRDDFunctions.saveAsHadoopDataset(PairRDDFunctions.scala:1094)
	at org.apache.spark.rdd.PairRDDFunctions$$anonfun$saveAsHadoopFile$4.apply$mcV$sp(PairRDDFunctions.scala:1067)
	at org.apache.spark.rdd.PairRDDFu

In [4]:
# dataset_1_data = sc.read.csv(dataset1_path, header=True)
# dataset_2_data = sc.read.csv(dataset2_path, header=True)
# dataset_3_data = sc.read.csv(dataset3_path, header=True)
# dataset_4_data = sc.read.csv(dataset4_path, header=True)
data = sc.textFile(dataset5_path)
# dataset_5_data = sc.read.csv(dataset5_path, header=True)




In [5]:
words = data.flatMap(lambda line: line.split(" "))
wordscount = words.map(lambda word : (word, 1)).reduceByKey(lambda a, b : a + b)

In [4]:
sc

<SparkContext master=local appName=My app>

In [35]:
dataset_1_data = dataset_1_data.withColumn("Year", lit("2015"))

In [14]:
dataset_2_data = dataset_2_data.withColumn("Year", lit("2016"))
dataset_3_data = dataset_3_data.withColumn("Year", lit("2017"))
dataset_4_data = dataset_4_data.withColumn("Year", lit("2018"))
# dataset_5_data = dataset_5_data.withColumn("Year", lit("2019"))

In [36]:

dataset_1_data.toPandas()

,Country,Region,Happiness_Rank,Happiness_Score,Standard_Error,Economy (GDP per Capita),Family,Health(Life_Expectancy),Freedom,Trust(Government_Corruption),Generosity,Dystopia_Residual,Year
0,Switzerland,Western Europe,1,7.587,0.03411,1.39651,1.34951,0.94143,0.66557,0.41978,0.29678,2.51738,2015
1,Iceland,Western Europe,2,7.561,0.04884,1.30232,1.40223,0.94784,0.62877,0.14145,0.4363,2.70201,2015
2,Denmark,Western Europe,3,7.527,0.03328,1.32548,1.36058,0.87464,0.64938,0.48357,0.34139,2.49204,2015
3,Norway,Western Europe,4,7.522,0.0388,1.459,1.33095,0.88521,0.66973,0.36503,0.34699,2.46531,2015
4,Canada,North America,5,7.427,0.03553,1.32629,1.32261,0.90563,0.63297,0.32957,0.45811,2.45176,2015
5,Finland,Western Europe,6,7.406,0.0314,1.29025,1.31826,0.88911,0.64169,0.41372,0.23351,2.61955,2015
6,Netherlands,Western Europe,7,7.378,0.02799,1.32944,1.28017,0.89284,0.61576,0.31814,0.4761,2.4657,2015
7,Sweden,Western Europe,8,7.364,0.03157,1.33171,1.28907,0.91087,0.6598,0.43844,0.36262,2.37119,2015
8,New Zealand,Australia and New Zealand,9,7.286,0.03371,1.25018,1.31967,0.90837,0.63938,0.42922,0.47501,2.26425,2015
9,Australia,Australia and New Zealand,10,7.284,0.04083,1.33358,1.30923,0.93156,0.65124,0.35637,0.43562,2.26646,2015


In [37]:
for col in list(dataset_1_data.columns):
    print(col, dataset_1_data.filter((dataset_1_data[col] == "") | dataset_1_data[col].isNull()).count())

Country 0
Region 0
Happiness_Rank 0
Happiness_Score 0
Standard_Error 0
Economy (GDP per Capita) 0
Family 0
Health(Life_Expectancy) 0
Freedom 0
Trust(Government_Corruption) 0
Generosity 0
Dystopia_Residual 0
Year 0


In [15]:
for col in list(dataset_2_data.columns):
    print(col, dataset_2_data.filter((dataset_2_data[col] == "") | dataset_2_data[col].isNull()).count())

Country 0
Region 0
Happiness Rank 0
Happiness Score 0
Lower Confidence Interval 0
Upper Confidence Interval 0
Economy (GDP per Capita) 0
Family 0
Health (Life Expectancy) 0
Freedom 0
Trust (Government Corruption) 0
Generosity 0
Dystopia Residual 0
Year 0


In [38]:
dataset_1_data.createOrReplaceTempView('dataset_1_data')

In [52]:
happiness_2015 = sc.sql(""" SELECT Country, Region, Happiness_Score
                                FROM dataset_1_data 
                                WHERE Happiness_Score > 6
                                
                                ORDER BY Happiness_Score DESC""")
happiness_2015.limit(10).toPandas()

,Country,Region,Happiness_Score
0,Switzerland,Western Europe,7.587
1,Iceland,Western Europe,7.561
2,Denmark,Western Europe,7.527
3,Norway,Western Europe,7.522
4,Canada,North America,7.427
5,Finland,Western Europe,7.406
6,Netherlands,Western Europe,7.378
7,Sweden,Western Europe,7.364
8,New Zealand,Australia and New Zealand,7.286
9,Australia,Australia and New Zealand,7.284


In [57]:
happiness_2015 = sc.sql(""" SELECT Country, Region, Happiness_Score
                                FROM dataset_1_data 
                                ORDER BY Happiness_Score""")
happiness_2015.limit(10).toPandas()

,Country,Region,Happiness_Score
0,Togo,Sub-Saharan Africa,2.839
1,Burundi,Sub-Saharan Africa,2.905
2,Syria,Middle East and Northern Africa,3.006
3,Benin,Sub-Saharan Africa,3.34
4,Rwanda,Sub-Saharan Africa,3.465
5,Afghanistan,Southern Asia,3.575
6,Burkina Faso,Sub-Saharan Africa,3.587
7,Ivory Coast,Sub-Saharan Africa,3.655
8,Guinea,Sub-Saharan Africa,3.656
9,Chad,Sub-Saharan Africa,3.667


In [45]:
happiness_2015.write.format('com.databricks.spark.csv').save('./my.csv',header = 'true')

In [63]:
sc.stop()

In [59]:
#MongoDB tutorial

In [98]:
from pymongo import MongoClient
client = pymongo.MongoClient("mongodb+srv://dbUser:<pass>@cluster0.t72w2.mongodb.net/myfirstdatabase?retryWrites=true&w=majority")
db = client.test


Collection(Database(MongoClient(host=['cluster0-shard-00-01.t72w2.mongodb.net:27017', 'cluster0-shard-00-00.t72w2.mongodb.net:27017', 'cluster0-shard-00-02.t72w2.mongodb.net:27017'], document_class=dict, tz_aware=False, connect=True, retrywrites=True, w='majority', authsource='admin', replicaset='atlas-o9qwvt-shard-0', ssl=True), 'test'), 'test-collection')

In [99]:
mydb = client["firstdatabase"]
mycol = mydb["customers"]

mydict = { "name": "John", "address": "Highway 37" }



In [100]:
mydict = { "name": "John", "address": "Highway 37" }

In [101]:
x = mycol.insert_one(mydict)

In [102]:
post_1 = {
    'title': 'Python and MongoDB',
    'content': 'PyMongo is fun, you guys',
    'author': 'Scott'
}
post_2 = {
    'title': 'Virtual Environments',
    'content': 'Use virtual environments, you guys',
    'author': 'Scott'
}
post_3 = {
    'title': 'Learning Python',
    'content': 'Learn Python, it is easy',
    'author': 'Bill'
}
new_result = mycol.insert_many([post_1, post_2, post_3])
print('Multiple posts: {0}'.format(new_result.inserted_ids))

Multiple posts: [ObjectId('5f2bcdfdd8def81850e9e781'), ObjectId('5f2bcdfdd8def81850e9e782'), ObjectId('5f2bcdfdd8def81850e9e783')]


In [95]:
bills_post = mycol.find_one({'author': 'Bill'})
print(bills_post)

{'_id': ObjectId('5f2bcca4d8def81850e9e77d'), 'title': 'Learning Python', 'content': 'Learn Python, it is easy', 'author': 'Bill'}


In [ ]:
mycol = mydb["customers"]

In [3]:
sc.stop()

In [4]:
from pyspark.sql import SQLContext
# conf = SparkConf().setAppName("test").setMaster("local")
# sc = SparkContext(conf=conf)
conf = SparkConf()
sc = SparkContext(conf=conf)

 

sqlContext = SQLContext(sc)

In [5]:
df1 = sqlContext.read.format('com.databricks.spark.csv').options(header='false', inferschema='true').load('2013_data_tiny.csv')

In [6]:
df1

DataFrame[_c0: int, _c1: int, _c2: int, _c3: int, _c4: int, _c5: int, _c6: int, _c7: string, _c8: int, _c9: string, _c10: string, _c11: string]

In [7]:

df1.show()

+----+---+---+---+---+---+-----+---+----+-------------+---------+----+
| _c0|_c1|_c2|_c3|_c4|_c5|  _c6|_c7| _c8|          _c9|     _c10|_c11|
+----+---+---+---+---+---+-----+---+----+-------------+---------+----+
|2013|  1|  1|  0|  0| 10|42101| co| 628| C628_42101_1|235.77068|null|
|2013|  1|  1|  0|  0| 10|42101| co|1035|C1035_42101_1|229.49535|null|
|2013|  1|  1|  0|  5| 10|42101| co| 628| C628_42101_1|221.14418|null|
|2013|  1|  1|  0|  5| 10|42101| co|1035|C1035_42101_1|231.42252|null|
|2013|  1|  1|  0| 10| 10|42101| co| 628| C628_42101_1|357.65799|null|
|2013|  1|  1|  0| 10| 10|42101| co|1035|C1035_42101_1|233.34969|null|
|2013|  1|  1|  0| 15| 10|42101| co| 628| C628_42101_1| 89.50581|null|
|2013|  1|  1|  0| 15| 10|42101| co|1035|C1035_42101_1|229.97713|null|
|2013|  1|  1|  0| 20| 10|42101| co| 628| C628_42101_1| 84.63032|null|
|2013|  1|  1|  0| 20| 10|42101| co|1035|C1035_42101_1|229.01355|null|
|2013|  1|  1|  0| 25| 10|42101| co| 628| C628_42101_1|152.88725|null|
|2013|

In [8]:
df = df1.selectExpr(" _c0 as year", "_c1 as month","_c2 as day","_c3 as hour", "_c4 as min", "_c5 as region","_c6 as parmater_id", "_c7 as name","_c8 as site","_c9 as cams","_c10 as value","_c11 as flag")
df.show()
df.printSchema()

df.coalesce(1).write.format('json').save('file_name.json')

+----+-----+---+----+---+------+-----------+----+----+-------------+---------+----+
|year|month|day|hour|min|region|parmater_id|name|site|         cams|    value|flag|
+----+-----+---+----+---+------+-----------+----+----+-------------+---------+----+
|2013|    1|  1|   0|  0|    10|      42101|  co| 628| C628_42101_1|235.77068|null|
|2013|    1|  1|   0|  0|    10|      42101|  co|1035|C1035_42101_1|229.49535|null|
|2013|    1|  1|   0|  5|    10|      42101|  co| 628| C628_42101_1|221.14418|null|
|2013|    1|  1|   0|  5|    10|      42101|  co|1035|C1035_42101_1|231.42252|null|
|2013|    1|  1|   0| 10|    10|      42101|  co| 628| C628_42101_1|357.65799|null|
|2013|    1|  1|   0| 10|    10|      42101|  co|1035|C1035_42101_1|233.34969|null|
|2013|    1|  1|   0| 15|    10|      42101|  co| 628| C628_42101_1| 89.50581|null|
|2013|    1|  1|   0| 15|    10|      42101|  co|1035|C1035_42101_1|229.97713|null|
|2013|    1|  1|   0| 20|    10|      42101|  co| 628| C628_42101_1| 84.6303